In [ ]:
!pip install tensorflow opencv-python mediapipe matplotlib numpy

In [1]:
import cv2
import numpy as np
import os
import time
import mediapipe as mp
import yaml
import shutil
import constants as cs
import globali as gg

In [2]:
def read_yaml_fields():
    with open(cs.YAMLFILE, 'r') as ff:
        content = yaml.safe_load(ff)
        a = set(content.get('actions', []))
        c = content.get('counter', {})
        return a, c

def update_yaml_fields(new_actions, new_counter):
    with open(cs.YAMLFILE, 'r') as ff:
        content = yaml.safe_load(ff)
    content['actions'] = new_actions
    content['counter'] = new_counter
    with open(cs.YAMLFILE, 'w') as ff:
        yaml.safe_dump(content, ff)
        
def reset_yaml_fields():
    current_timestamp_time = time.time()
    file_name, file_extension = os.path.splitext(cs.YAMLFILE)
    destination = ("backups/" +
                   file_name + "_" +
                   str(current_timestamp_time) +
                   file_extension)
    shutil.copy(cs.YAMLFILE, destination)
    content = {'actions': set(), 'counter': {}}
    with open(cs.YAMLFILE, 'w') as ff:
        yaml.safe_dump(content, ff)
        

In [3]:
gg.actions, gg.counters = read_yaml_fields()

In [4]:
# reset_yaml_fields()

In [5]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                       # Image is no longer writeable
    results = model.process(image)                      # Make prediction
    image.flags.writeable = True                        # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)      # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


In [7]:
# function that erase content videos' folder
def clean_folder():
    for file in os.listdir(cs.VIDEOS_FOLDER):
        file_path = os.path.join(cs.VIDEOS_FOLDER, file)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)


In [8]:
clean_folder()

In [9]:

def collect_data(action, no_sequences, no_frames, time_record, signer, camId):
    gg.actions, gg.counters = read_yaml_fields()
    
    if action not in gg.actions:
        gg.counters[action] = 0
    
    action_folder = os.path.join(cs.KEYPOINTS_FOLDER, action)
    os.makedirs(action_folder, exist_ok=True)
    action_folder2 = os.path.join(cs.VIDEOS_FOLDER, action)
    os.makedirs(action_folder2, exist_ok=True)
    gg.actions.add(action)
    
    # clean folder videos
    clean_folder()
    
    cap = cv2.VideoCapture(camId)
    
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        for sequence in range(no_sequences):
            
            video_folder = os.path.join(action_folder, str(gg.counters[action] + sequence))
            os.makedirs(video_folder, exist_ok=True)
            video_folder2 = os.path.join(action_folder2, str(gg.counters[action] + sequence))
            os.makedirs(video_folder2, exist_ok=True)
            
            for frame_num in range(no_frames):
                
                ret, frame = cap.read()                                                 # Read feed
                image, results = mediapipe_detection(frame, holistic)                   # Make detections
                draw_styled_landmarks(image, results)                                   # Draw landmarks     
                
                num = gg.counters[action] + sequence
                frame_path = os.path.join(cs.VIDEOS_FOLDER, action, str(num), str(frame_num) + ".jpg")
                cv2.imwrite(frame_path, frame)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(time_record)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = extract_keypoints(results)
                
                npy_path = os.path.join(video_folder, str(frame_num))
                np.save(npy_path, keypoints)
                
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
        # update actions and counters
        gg.counters[action] += no_sequences
        update_yaml_fields(gg.actions, gg.counters)
        
        cap.release()
        cv2.destroyAllWindows()
    cap.release()
    cv2.destroyAllWindows()    
    

In [10]:
words = ['zero', 'uno', 'due', 'tre', 'quattro', 'cinque', 'sei', 'sette', 'otto', 'nove',
         'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i','j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 
         'yes', 'no', 'see you later', 'hello', 'I_me', 'name', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''
         ]


In [22]:
collect_data(words[35], 50, cs.NUM_FRAME, 2000, "paolo", 0)                         # 35 Z

In [23]:
len(gg.actions)

36

In [24]:
gg.counters[("z")]

100